# Project Part 2 Solutions

## (a)

In [49]:
import numpy as np

file=np.load("part2.npz")
beta_old=file['beta_old']
N=file['N']
Svc_0_PMF=file['Svc_0_PMF']
Lc=file['Lc']
Ic_0=file['Ic_0']
gamma=file['gamma']
L_observed=file['L_observed']
nb_nodes = Svc_0_PMF.shape[0]

In [50]:
def SIR(t, Z, N, betas, gamma):
    (S_vec, I_vec, R) = (Z[:16].reshape(4, 4), Z[16:20], Z[-1])
    I = I_vec.sum()
    dSdt = -betas*S_vec*I/N 
    dIdt = -dSdt.sum(axis=0)-gamma*I_vec
    dRdt = np.array([gamma*I])
    return np.concatenate((dSdt.flatten(), dIdt, dRdt))

In [61]:
from scipy.integrate import solve_ivp
from scipy.optimize import fmin
from sklearn.metrics import mean_squared_error
import copy 
Svc_0_PMF /= Svc_0_PMF.sum(axis=(1, 2), keepdims=True)
Svc_0 = (N-Ic_0.sum(axis=1))[:, np.newaxis, np.newaxis]*Svc_0_PMF
Svc_0 = Svc_0.round().astype(int)
R0 = np.zeros((nb_nodes)).astype(int)

t = np.linspace(0, 199, 200)

S = np.zeros((nb_nodes, 4, 4, len(t)))
I = np.zeros((nb_nodes, 4, len(t)))
R = np.zeros((nb_nodes, len(t)))
L = np.zeros((nb_nodes, len(t)))

for N_node, node_index in zip(N, range(nb_nodes)):

    initial_conditions = np.concatenate(
        (Svc_0[node_index].flatten(), Ic_0[node_index], [R0[node_index]]))  

    soln = solve_ivp(
        SIR, (t[0], t[9]), initial_conditions, args=(N_node, beta_old, gamma), t_eval=t[:10])

    S[node_index, :, :, :10] = soln.y[:16].reshape(4, 4, -1)
    I[node_index, :, :10] = soln.y[16:20]
    R[node_index, :10] = soln.y[20]
    L[node_index, :10] = Lc@soln.y[16:20]


alpha_predicted = np.zeros((nb_nodes))


def loss_func(alpha):   
    beta_new = beta_old * alpha     
    day_9_initial = np.concatenate( 
    (S[node_index, :, :, 9].flatten(), I[node_index, :, 9], [R[node_index, 9]]))  

    day10_29_pred = solve_ivp( 
        SIR, (t[9], t[30]), day_9_initial, args = (N_node, beta_new, gamma), t_eval = t[9:30])    
    L_pred = Lc@day10_29_pred.y[16:20] #day 9 is stuck in there i made it 21 values for a reason and then delete day 9     
    #L_pred = np.delete(L_pred,0)
    L_actual = L_observed[node_index][9:30]   
    mse = mean_squared_error(L_actual,L_pred)
    return mse


for N_node, node_index in zip(N, range(nb_nodes)):  
    alpha_predicted[node_index] = fmin(loss_func, 0.5, disp=False,ftol=1e-6)
alpha_copy = copy.deepcopy(alpha_predicted)
print("Node index ranked from highest predicted alpha value to lowest predicted alpha value:\n")  
for i in range(100):    
    index = np.argmax(alpha_copy)
    print(str(index)+" : "+str(alpha_copy[index])) 
    alpha_copy[index] = -1000 
  


Node index ranked from highest predicted alpha value to lowest predicted alpha value:

88 : 0.9532714843750012
97 : 0.9282318115234385
39 : 0.8635772705078135
3 : 0.7097541809082037
82 : 0.698361206054688
46 : 0.6943862915039066
99 : 0.6838226318359379
4 : 0.6778827667236333
29 : 0.6604969024658207
55 : 0.6509254455566411
77 : 0.645151519775391
35 : 0.6384799003601078
34 : 0.5956703186035159
18 : 0.5807563781738283
76 : 0.5627861022949219
59 : 0.5542316436767578
17 : 0.5529579162597658
87 : 0.536046600341797
57 : 0.5210838317871094
65 : 0.5083087921142578
89 : 0.5070587158203126
81 : 0.5023948669433593
22 : 0.48616943359375003
28 : 0.47091369628906243
68 : 0.4564775466918945
73 : 0.44996871948242184
94 : 0.4487113952636718
41 : 0.4375556945800781
45 : 0.43438644409179683
42 : 0.42476272583007807
67 : 0.41876068115234366
63 : 0.4178750991821289
40 : 0.4144542694091795
74 : 0.4092250823974609
36 : 0.39033279418945305
1 : 0.386005687713623
32 : 0.38393640518188465
8 : 0.3794620513916014
7

## (b)

In [ ]:
def g(money):
    alpha = 1/np.log2(2*10**(-3)*money+2)
    return alpha

def g_inv(alpha):
    money = (2**(1/alpha)-2)/(2*10**(-3))
    return money
    import matplotlib.pyplot as plt 

def scatter_plot(alpha_afterNPI):
    plt.figure()
    colors=(beta_old[np.newaxis,:,:]*Svc_0_PMF).mean(axis=(1,2))
    plt.scatter(alpha_predicted, alpha_afterNPI, s=N/500, c=colors, cmap='jet', alpha=0.5)
    plt.xlabel(r"$\alpha_{own}$")
    plt.ylabel(r"$\alpha_{after\ NPI}$")

def total_recovered_difference(alpha_afterNPI):
    
    ###### WRITE YOUR FUNCTION HERE
    
    return difference

### Policy #1

i.

ii.

### Policy #2

i. 

ii.

### Policy #3

i.

ii.

### Policy #4

i.

ii.

## (c)

### Policy 1

### Policy 2

### Policy 3

### Policy 4